In [175]:
import pandas as pd
import glob
import numpy as np
Names = glob.glob(r'/Users/ortrabelsi/Desktop/מטלה דימות נתונים/סופי 1/Training' + "/*")
li = []
for i,name in enumerate(Names):
   
    all_files = glob.glob(name + "/*.csv")
    labels = ["Spontan", "Sync" , "Alone"]

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        for j,label in enumerate(labels):
            if label in filename:
                df["Label"] = j
                df["Name"] = i
                df = (df[df["Time"] > (df["Time"][0]+7)]) # remove the first 7 sec from each test
        li.append(df)

# merge data frames.
frame = pd.concat(li, axis=0, ignore_index=True)

# Make hand type to 0 or 1.
frame.loc[(frame[' Hand Type'] == 'left') , " Hand Type"] = int(0)
frame.loc[(frame[' Hand Type'] == 'right') , " Hand Type"] = int(1)
frame[' Hand Type'] = frame[' Hand Type'].astype(np.int)

### We index all the names. Because we want when all the information is numerical:
  
### Names:  
0 Oriya Social_Nuero  
1 Evyatar Cohen  
2 Shelly Social_Nuero  
3 Revital Marbel  
4 Nofar Social_Nuero  
5 Yoel Raz  
6 Orya Kalmanovitz  
7 Shahar Terner  
8 Yael Hagai  
  
### Labels:    
0 Spontan  
1 Sync  
2 Alone  
  
### Hand Type:    
0 left  
1 right

In [176]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172916 entries, 0 to 172915
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Time             172916 non-null  float64
 1    Frame ID        172916 non-null  int64  
 2    Hand Type       172916 non-null  int64  
 3    # hands         172916 non-null  int64  
 4    Position X      172916 non-null  float64
 5    Position Y      172916 non-null  float64
 6    Position Z      172916 non-null  float64
 7    Velocity X      172916 non-null  float64
 8    Velocity Y      172916 non-null  float64
 9    Velocity Z      172916 non-null  float64
 10   Pitch           172916 non-null  float64
 11   Roll            172916 non-null  float64
 12   Yaw             172916 non-null  float64
 13   Wrist Pos X     172916 non-null  float64
 14   Wrist Pos Y     172916 non-null  float64
 15   Wrist Pos Z     172916 non-null  float64
 16   Elbow pos X     172916 non-null  floa

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,Label,Name


In [98]:
frame.isna().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
Label              0
Name               0
dtype: int64

In [64]:
frame[(frame[" # hands"] == 1) & (frame["Label"] != "Alone") ]

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,Label,Name
13484,773.7871,89259,left,1,-162.0288,174.2548,74.048190,-137.642900,183.36750,-28.616020,...,118.2664,94.38357,-408.7272,-38.65267,109.12010,0.0,0.000000,0.0,Sync,Oriya Social_Nuero
13485,773.8040,89261,left,1,-164.4983,177.5058,73.821880,-148.584000,194.20040,-6.854675,...,120.9421,93.96440,-410.1612,-36.72564,104.49990,0.0,0.000000,0.0,Sync,Oriya Social_Nuero
13486,773.8209,89263,left,1,-167.7155,181.1708,74.157640,-188.106900,206.96150,28.999040,...,124.2479,94.19987,-412.7216,-33.89042,100.16370,0.0,0.000000,0.0,Sync,Oriya Social_Nuero
13487,773.8369,89265,left,1,-170.9337,184.6969,74.834000,-189.325800,201.02180,43.616890,...,127.2255,94.67654,-414.0762,-32.46870,98.70818,0.0,0.000000,0.0,Sync,Oriya Social_Nuero
13488,773.8538,89267,left,1,-174.6012,188.2105,76.107360,-218.362400,202.17340,84.412640,...,130.1557,95.64534,-414.9131,-32.24659,98.66123,0.0,0.000000,0.0,Sync,Oriya Social_Nuero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189713,240.1515,28216,right,1,182.6603,265.2973,0.782887,14.271670,9.27431,14.907360,...,218.1339,37.19538,377.6131,37.14408,104.58710,0.0,0.190795,0.0,Spontan,Yael Hagai
189714,240.1680,28218,right,1,182.7662,265.2484,0.794763,1.387123,-21.97268,-0.809654,...,217.5685,37.06472,374.9344,34.20661,103.27690,0.0,0.206666,0.0,Spontan,Yael Hagai
189715,240.1846,28219,right,1,182.8512,264.9661,0.707451,9.984365,-33.16511,-10.258710,...,217.2337,37.02989,373.0332,32.43384,103.57720,0.0,0.217796,0.0,Spontan,Yael Hagai
189716,240.2012,28221,right,1,182.8170,263.8030,0.159580,0.880788,-63.79849,-44.794880,...,216.9306,36.64636,372.0320,31.82113,106.58360,0.0,0.231684,0.0,Spontan,Yael Hagai
